**Table-aware plot embedding**

It should be possible to embed a bar plot or a line plot more easily than the current approach:

* The data for the plot would come from table columns, instead of separately-supplied series.
    * This would eliminate the awkward "reversal" of series data.
* The layout of the embedded plot should be responsive to cell spacing, gaps, padding, etc.
    * This would eliminate awkward manual padding to align vis datums with cells.
* Does the table code render the graphics itself, or act as a constructor for an existing vis type?
    * Hopefully the latter, to avoid code repetition.
    * toyplot.projection.Piecewise might already be able to handle complicated cell spacing.
    * The actual axes would be constructed lazily in _finalize().
* Managing the relationship(s) among merged cells, formatters, and plotting could be tricky.
    * Perhaps the new plotting code replaces the current merge + create axes approach:
    
            # Instead of this:
            axes = <cells>.merge().axes(...)
            axes.bars(...)
            
            # Do this:
            <cells>.bars(...)

**Odds-and-ends**

* Clear cell data when embedding axes.
* Do a better job updating gaps when we insert / delete cells.
* Should grid lines be rendered down the middle of a gap (as we do today), or to either side?

In [1]:
import numpy
import toyplot.data

import logging
toyplot.log.setLevel(logging.DEBUG)

In [2]:
canvas, table = toyplot.table(trows=3, rows=3, brows=3, lcolumns=3, columns=3, rcolumns=3, filename="rainbow")
table.cells.cells.data = numpy.arange(81).reshape(9, 9)
table.cells.grid.hlines[...] = "single"
table.cells.grid.vlines[...] = "single"
table.top.left.cells.style = {"fill":"red", "opacity":0.3}
table.top.cells.style = {"fill":"orange", "opacity":0.3}
table.top.right.cells.style = {"fill":"yellow", "opacity":0.3}
table.right.cells.style = {"fill":"greenyellow", "opacity":0.3}
table.bottom.right.cells.style = {"fill":"green", "opacity":0.3}
table.bottom.cells.style = {"fill":"aqua", "opacity":0.3}
table.bottom.left.cells.style = {"fill":"blue", "opacity":0.3}
table.left.cells.style = {"fill":"purple", "opacity":0.3}

numpy.random.seed(1234)
axes = table.left.column[1].merge().cartesian(cell_padding=5)
axes.bars(numpy.random.normal(loc=5, size=(20, 2)), along="y", opacity=0.5);
axes = table.right.column[1].merge().cartesian(cell_padding=5)
axes.plot(numpy.random.normal(loc=5, size=(20, 3)), along="y");
table.cells.column[4].bars()
table.cells.row[0:3].height = 25
table.cells.row[4].height = "1in"
table.cells.column[4].width = "1in"

DEBUG:toyplot:[ -50.   -75.  -100.  -125.  -190.8 -286.8 -352.6 -418.4 -484.2]
DEBUG:toyplot:[ -75.  -100.  -125.  -190.8 -286.8 -352.6 -418.4 -484.2 -550. ]
DEBUG:toyplot:[4 13 22 31 40 49 58 67 76]


0 1 2 3 5 6 7 8 9 10 11 12 14 15 16 17 18 19 20 21 23 24 25 26 27 29 30 32 33 35 36 38 39 41 42 44 45 47 48 50 51 53 54 55 56 57 59 60 61 62 63 64 65 66 68 69 70 71 72 73 74 75 77 78 79 80 4 <path d="M 480.35061468934748 347.60000000000002 L 464.77664206240189 336.1473684210527 L 478.8085895593058 324.69473684210527 L 481.35743862921117 313.2421052631579 L 469.64902119544968 301.78947368421052 L 478.75194091288222 290.3368421052632 L 483.89474979738634 278.88421052631583 L 479.28967024562678 267.43157894736839 L 487.28340204638721 255.97894736842107 L 471.52659088771486 244.5263157894737 L 474.11259912514015 233.07368421052632 L 483.58901295052181 221.62105263157895 L 481.44209991511127 210.16842105263157 L 480.08501090866906 198.7157894736842 L 489.36341591319695 187.26315789473685 L 476.68516636928723 175.81052631578947 L 481.7597314357638 164.35789473684213 L 476.90330790019635 152.90526315789472 L 485.71088032702784 141.45263157894738 L 483.62362021227239 130.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 475.69802550969018 347.60000000000002 L 481.59290131043349 336.1473684210527 L 479.93135823082332 324.69473684210527 L 485.88763772940445 313.2421052631579 L 479.06832720813117 301.78947368421052 L 482.361965818844 290.3368421052632 L 490.9891235675222 278.88421052631583 L 482.028564904316 267.43157894736839 L 462.42027883132687 255.97894736842107 L 481.332930689112 244.5263157894737 L 483.14414347696351 233.07368421052632 L 473.09310484639491 221.62105263157895 L 471.45399203413155 210.16842105263157 L 483.1610308166604 198.7157894736842 L 480.90395532221385 187.26315789473685 L 485.35450253625686 175.81052631578947 L 489.87657316824345 164.35789473684213 L 472.35757401730689 152.90526315789472 L 479.20567346818518 141.45263157894738 L 472.05974379190957 130.0" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> <path d="M 480.06060754075634 347.60000000000002 L 473.30376351458108 336.1473684210527 L 485.26795317970254 324.69473684210527 L 469.33165400808457 313.2421052631579 L 475.82981405814274 301.78947368421052 L 479.54184394371936 290.3368421052632 L 472.74572729249405 278.88421052631583 L 478.28947144732558 267.43157894736839 L 494.5 255.97894736842107 L 484.90094623330481 244.5263157894737 L 484.89738139153854 233.07368421052632 L 494.33521857037704 221.62105263157895 L 484.37431870630382 210.16842105263157 L 454.0 198.7157894736842 L 480.99010097161568 187.26315789473685 L 486.92951690063751 175.81052631578947 L 480.37698708786922 164.35789473684213 L 475.56573887032653 152.90526315789472 L 477.30293140280571 141.45263157894738 L 476.09305941189587 130.0" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:1.0;stroke-width:2.0"> 0 25 50 75 
 
 
 Save as .csv